In [266]:
import numpy as np
import pandas
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

In [267]:
data = pd.read_csv('./train.csv').iloc[:, 1:]
test = pd.read_csv('./test.csv')
ids = test.iloc[:,0]
test = test.iloc[:,1:]
data.shape

# test.shape
# type(data)

(750000, 11)

In [268]:
# 查看data 各列的类型
data.dtypes

Podcast_Name                    object
Episode_Title                   object
Episode_Length_minutes         float64
Genre                           object
Host_Popularity_percentage     float64
Publication_Day                 object
Publication_Time                object
Guest_Popularity_percentage    float64
Number_of_Ads                  float64
Episode_Sentiment               object
Listening_Time_minutes         float64
dtype: object

In [269]:
test.dtypes

Podcast_Name                    object
Episode_Title                   object
Episode_Length_minutes         float64
Genre                           object
Host_Popularity_percentage     float64
Publication_Day                 object
Publication_Time                object
Guest_Popularity_percentage    float64
Number_of_Ads                  float64
Episode_Sentiment               object
dtype: object

In [235]:
# 删除 博客名称、上映时间这两列
# data.drop(columns=['Podcast_Name','Episode_Title'], inplace=True)
# test.drop(columns=['Podcast_Name','Episode_Title'], inplace=True)
# test.shape

In [236]:
# 查看前十列数据
data.head(10)

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031
5,Fitness First,Episode 19,26.54,Health,48.96,Saturday,Afternoon,NaN,3.0,Positive,22.77047
6,Criminal Minds,Episode 47,69.83,True Crime,35.82,Sunday,Night,39.02,0.0,Neutral,64.75024
7,News Roundup,Episode 44,48.52,News,44.99,Thursday,Night,20.12,0.0,Positive,22.37517
8,Daily Digest,Episode 32,105.87,News,69.81,Monday,Evening,NaN,2.0,Neutral,68.00124
9,Music Matters,Episode 81,NaN,Music,82.18,Thursday,Night,59.72,3.0,Neutral,45.94761


In [237]:
data.dtypes

Podcast_Name                    object
Episode_Title                   object
Episode_Length_minutes         float64
Genre                           object
Host_Popularity_percentage     float64
Publication_Day                 object
Publication_Time                object
Guest_Popularity_percentage    float64
Number_of_Ads                  float64
Episode_Sentiment               object
Listening_Time_minutes         float64
dtype: object

In [226]:
# data.dropna(inplace=True)
# test.dropna(inplace=True)
# 数值型
data['Episode_Length_minutes'].fillna(data['Episode_Length_minutes'].median(), inplace=True)
data['Host_Popularity_percentage'].fillna(data['Host_Popularity_percentage'].median(), inplace=True)
data['Guest_Popularity_percentage'].fillna(data['Guest_Popularity_percentage'].median(), inplace=True)
data['Number_of_Ads'].fillna(data['Number_of_Ads'].mode()[0], inplace=True)

# 类别型
data['Genre'].fillna('Unknown', inplace=True)
data['Episode_Sentiment'].fillna(data['Episode_Sentiment'].mode()[0], inplace=True)
data['Publication_Day'].fillna(data['Publication_Day'].mode()[0], inplace=True)
data['Publication_Time'].fillna(data['Publication_Time'].mode()[0], inplace=True)


C:\Users\21996\AppData\Local\Temp\ipykernel_37280\188880239.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Episode_Length_minutes'].fillna(data['Episode_Length_minutes'].median(), inplace=True)
C:\Users\21996\AppData\Local\Temp\ipykernel_37280\188880239.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

In [281]:
data.head(10)

,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes,Podcast_Name_Athlete's Arena,Podcast_Name_Brain Boost,Podcast_Name_Business Briefs,Podcast_Name_Business Insights,Podcast_Name_Comedy Corner,...,Publication_Day_Thursday,Publication_Day_Tuesday,Publication_Day_Wednesday,Publication_Time_Afternoon,Publication_Time_Evening,Publication_Time_Morning,Publication_Time_Night,Episode_Sentiment_Negative,Episode_Sentiment_Neutral,Episode_Sentiment_Positive
0,44.605046,74.81,NaN,0.0,31.41998,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,True
1,119.800000,66.95,75.95,2.0,88.01241,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
2,73.900000,69.97,8.97,0.0,44.92531,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
3,67.170000,57.22,78.70,2.0,46.27824,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
4,110.510000,80.07,58.68,3.0,75.61031,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
5,26.540000,48.96,NaN,3.0,22.77047,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
6,69.830000,35.82,39.02,0.0,64.75024,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
7,48.520000,44.99,20.12,0.0,22.37517,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,True
8,105.870000,69.81,NaN,2.0,68.00124,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
9,65.229044,82.18,59.72,3.0,45.94761,False,False,False,False,False,...,True,False,False,False,False,False,True,False,True,False


In [258]:
x_train = data['Listening_Time_minutes'].values.reshape(-1, 1)
y_train = data['Episode_Length_minutes'].values.reshape(-1, 1)
x_train.shape

(750000, 1)

In [260]:
ratio = data['Listening_Time_minutes'].mean() / data['Episode_Length_minutes'].mean()

In [280]:
data['Episode_Length_minutes'] = data['Episode_Length_minutes'].fillna(
    data['Listening_Time_minutes'] / ratio)

In [263]:
data['Episode_Length_minutes']

0          44.605046
1         119.800000
2          73.900000
3          67.170000
4         110.510000
             ...    
749995     75.660000
749996     75.750000
749997     30.980000
749998    108.980000
749999     24.100000
Name: Episode_Length_minutes, Length: 750000, dtype: float64

In [ ]:
test.isnull().sum()



In [228]:
test['Episode_Length_minutes'].fillna(test['Episode_Length_minutes'].median(), inplace=True)
test['Guest_Popularity_percentage'].fillna(test['Guest_Popularity_percentage'].median(), inplace=True)

C:\Users\21996\AppData\Local\Temp\ipykernel_37280\1860491479.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Episode_Length_minutes'].fillna(test['Episode_Length_minutes'].median(), inplace=True)
C:\Users\21996\AppData\Local\Temp\ipykernel_37280\1860491479.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

In [249]:
data['Episode_Length_minutes','Listening_Time_minutes']

KeyError: ('Episode_Length_minutes', 'Listening_Time_minutes')

In [272]:
correlearion = data.corr(method='pearson')


,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes,Podcast_Name_Athlete's Arena,Podcast_Name_Brain Boost,Podcast_Name_Business Briefs,Podcast_Name_Business Insights,Podcast_Name_Comedy Corner,...,Publication_Day_Thursday,Publication_Day_Tuesday,Publication_Day_Wednesday,Publication_Time_Afternoon,Publication_Time_Evening,Publication_Time_Morning,Publication_Time_Night,Episode_Sentiment_Negative,Episode_Sentiment_Neutral,Episode_Sentiment_Positive
Episode_Length_minutes,1.000000,0.023596,-0.009704,-0.054953,0.916749,0.006242,-0.003970,0.009997,-0.008176,-0.008350,...,-0.010973,0.014784,0.005512,0.002465,-0.010760,-0.012846,0.020763,-0.029895,0.014377,0.015550
Host_Popularity_percentage,0.023596,1.000000,0.022431,-0.017896,0.050870,-0.000756,0.005552,0.005049,0.002112,0.000126,...,-0.005139,-0.000852,0.001594,-0.002786,0.003912,-0.000166,-0.001044,-0.005764,-0.000712,0.006486
Guest_Popularity_percentage,-0.009704,0.022431,1.000000,0.007933,-0.016014,0.008967,-0.003298,-0.005024,0.000005,0.005193,...,-0.002154,0.000333,0.002755,0.006573,-0.002178,-0.000593,-0.003617,0.003105,-0.007074,0.003979
Number_of_Ads,-0.054953,-0.017896,0.007933,1.000000,-0.118337,-0.013759,0.003841,0.005494,0.005959,-0.004080,...,-0.001240,0.003936,-0.005025,-0.001749,0.012812,-0.003346,-0.007858,0.012815,0.008629,-0.021487
Listening_Time_minutes,0.916749,0.050870,-0.016014,-0.118337,1.000000,0.006785,-0.004735,0.009452,-0.007365,-0.008180,...,-0.008410,0.009920,0.005585,0.001823,-0.014801,-0.009719,0.022405,-0.034942,0.001614,0.033377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Publication_Time_Morning,-0.012846,-0.000166,-0.000593,-0.003346,-0.009719,-0.002196,0.002847,0.000725,-0.005262,0.002388,...,0.003677,0.007913,-0.006256,-0.312762,-0.331446,1.000000,-0.332673,-0.009659,0.000866,0.008806
Publication_Time_Night,0.020763,-0.001044,-0.003617,-0.007858,0.022405,0.005754,-0.002428,-0.002403,0.004995,0.004534,...,0.002532,-0.003825,-0.000789,-0.334569,-0.354556,-0.332673,1.000000,-0.002512,-0.003821,0.006347
Episode_Sentiment_Negative,-0.029895,-0.005764,0.003105,0.012815,-0.034942,0.001556,-0.001550,-0.000898,0.006893,0.001698,...,-0.002384,-0.005344,-0.000852,0.002216,0.009719,-0.009659,-0.002512,1.000000,-0.502112,-0.498064
Episode_Sentiment_Neutral,0.014377,-0.000712,-0.007074,0.008629,0.001614,-0.003168,-0.001481,-0.001858,-0.005681,-0.002790,...,-0.004402,0.002623,0.001926,-0.000589,0.003561,0.000866,-0.003821,-0.502112,1.000000,-0.499821


In [282]:
correlearion['Guest_Popularity_percentage']

Episode_Length_minutes        -0.009704
Host_Popularity_percentage     0.022431
Guest_Popularity_percentage    1.000000
Number_of_Ads                  0.007933
Listening_Time_minutes        -0.016014
                                 ...   
Publication_Time_Morning      -0.000593
Publication_Time_Night        -0.003617
Episode_Sentiment_Negative     0.003105
Episode_Sentiment_Neutral     -0.007074
Episode_Sentiment_Positive     0.003979
Name: Guest_Popularity_percentage, Length: 177, dtype: float64

In [271]:
# columns_object = ['Genre','Publication_Day','Publication_Time','Episode_Sentiment']
# for column in columns_object:
#     # 数据格式转化为分类格式
#     data[column] = data[column].astype('category')
#     test[column] = test[column].astype('category')
#     print(f'{column}:{data[column].cat.categories}')
#     # 将类别按照顺序转换为整数
#     data[column] = data[column].cat.codes.to_list()
#     test[column] = test[column].cat.codes.to_list()
data = pd.get_dummies(data)
test = pd.get_dummies(test)
data.head(10)

,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes,Podcast_Name_Athlete's Arena,Podcast_Name_Brain Boost,Podcast_Name_Business Briefs,Podcast_Name_Business Insights,Podcast_Name_Comedy Corner,...,Publication_Day_Thursday,Publication_Day_Tuesday,Publication_Day_Wednesday,Publication_Time_Afternoon,Publication_Time_Evening,Publication_Time_Morning,Publication_Time_Night,Episode_Sentiment_Negative,Episode_Sentiment_Neutral,Episode_Sentiment_Positive
0,NaN,74.81,NaN,0.0,31.41998,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,True
1,119.80,66.95,75.95,2.0,88.01241,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
2,73.90,69.97,8.97,0.0,44.92531,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
3,67.17,57.22,78.70,2.0,46.27824,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
4,110.51,80.07,58.68,3.0,75.61031,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
5,26.54,48.96,NaN,3.0,22.77047,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
6,69.83,35.82,39.02,0.0,64.75024,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
7,48.52,44.99,20.12,0.0,22.37517,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,True
8,105.87,69.81,NaN,2.0,68.00124,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
9,NaN,82.18,59.72,3.0,45.94761,False,False,False,False,False,...,True,False,False,False,False,False,True,False,True,False


In [ ]:
test.shape

In [ ]:
test.isnull().sum()

In [ ]:
test.mode()

In [ ]:
target = "Listening_Time_minutes"

In [ ]:
train_labels = data[target]


In [ ]:
data_train = data.drop(columns=[target]).astype(float)

In [ ]:
# test.isnull().sum()
label_mean = train_labels.mean()
label_std = train_labels.std()
# train_labels = (train_labels-label_mean)/label_std

In [ ]:
# min_max_scaler = preprocessing.MinMaxScaler()
# data_train=min_max_scaler.fit_transform(data_train)
# data_test=min_max_scaler.transform(test)

In [ ]:
test = test.astype(float)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data,mode, target=None):
        self.mode = mode
        if mode == "test":
            self.data = torch.tensor(data).float()
        elif mode == "train":
            self.data = torch.tensor(data).float()
            self.target = torch.tensor(target).float()
    def __getitem__(self, index):
        if self.mode == "test":
            return self.data[index]
        return self.data[index],self.target[index]
    def __len__(self):
        return len(self.data)

In [ ]:
train_set = MyDataset(data_train.values,"train",train_labels)
test_set = MyDataset(test.values,"test")

In [ ]:
train_loader = DataLoader(dataset=train_set,batch_size=2048,shuffle=True)
test_loader = DataLoader(dataset=test_set,batch_size=2048,shuffle=False)

In [ ]:
len(test_loader.dataset)

In [ ]:
class model(nn.Module):
    def __init__(self, input_size, output_size):
        super(model, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        return self.net(x)
    # def __init__(self,input_size,output_size):
    #     super(model, self).__init__()
    #     self.net = nn.Sequential(
    #         nn.Linear(input_size,2048),
    #         nn.ReLU(),
    #         nn.Linear(2048, output_size),
    #     )
    # def forward(self,x):
    #     x = self.net(x)
    #     return x

In [ ]:
input_size = data_train.shape[1]
output_size = 1
epochs =15
model = model(input_size,output_size).to("cuda")
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
outputs = []

In [ ]:
for epoch in range(epochs):
    model.train()
    loss_all=0.0
    for data,target in train_loader:
        data,target = data.to("cuda"),target.to("cuda")
        optimizer.zero_grad()
        output = model(data)
        output = output.squeeze(1)
        loss = loss_fn(output,target)
        loss_all += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, train loss {loss_all/len(train_loader)}")

In [ ]:
with torch.no_grad():
    model.eval()
    for data in test_loader:
        data = data.to("cuda")
        output = model(data)
        outputs.append(output)

In [ ]:
outputs_tensor = torch.cat(outputs, dim=0).cpu().detach().numpy().flatten()
df_results = pd.DataFrame({
    "id":ids,
    "Listening_Time_minutes": outputs_tensor   # 模型预测值
})
df_results.to_csv("test_predictions9.csv", index=False)
print("预测结果已保存为 test_predictions_with_ids6.csv")